<h1>Run cell below too</h1>
if window doesn't pop up by itself, check for a new window on app bar

In [3]:
from config import *

conf = Config()

print(type(conf))
print(conf.project_config)


<class 'config.Config'>
{'model_checkpoint_path': 'E:/absolute/path/where/models/will/be/saved', 'KITTI_dataset_path': 'E:/abosolute/path/to/root/of/dataset', 'saved_video_path': 'E:/absolute/path/to/directory/with/realsense/files', 'default_net_config': 'default_net', 'default_train_config': 'default_train', 'default_test_config': 'default_test'}


In [4]:
from tensorflow.keras.layers import Conv2D, Input, MaxPool2D, Concatenate, Lambda, Dense
from tensorflow.keras.layers import TimeDistributed as Parallel
from tensorflow.keras.models import Model
from functools import partial
import tensorflow.keras.backend as K
import tensorflow as tf

print(tf.version.VERSION)


def MLP(inp,shape):
    
    MLP = Dense(shape[0])(inp)
    for i in range(1,len(shape)):
        MLP = Dense(shape[i])(MLP)
        
    return MLP
        
        

    
inp = Input(shape=(200,21))

MLP = MLP(inp,(16,8,1))

topk = Lambda(lambda x: tf.math.top_k(x))(MLP)

model = Model(inp,topk)

model.summary()
        
    
        
        
    
    
    
    

2.2.0
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 21)]         0         
_________________________________________________________________
dense (Dense)                (None, 200, 16)           352       
_________________________________________________________________
dense_1 (Dense)              (None, 200, 8)            136       
_________________________________________________________________
dense_2 (Dense)              (None, 200, 1)            9         
_________________________________________________________________
lambda (Lambda)              TopKV2(values=(None, 200, 0         
Total params: 497
Trainable params: 497
Non-trainable params: 0
_________________________________________________________________


In [12]:
import numpy as np

x = np.random.random((10,3))
y=np.array([2,3,4])


print(np.tile(y,3))

[2 3 4 2 3 4 2 3 4]
